In [1]:
import numpy as np 
import pandas as pd
import os

from rpy2.robjects.packages import importr
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
import re

/home/mr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning:

Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.

[MLENS] backend: threading


In [3]:
n, p = 1000, 10

x = np.random.normal(0, .5, size=(n, p))

alpha = np.random.uniform(.2, .5, size=(p, 1))
beta = np.random.uniform(.2, .5, size=(p, 1))

xalpha = np.dot(x, alpha)
xbeta = np.dot(x, beta)

pr = 1/(1 + np.exp(-xalpha))
A = np.random.binomial(1, pr, size=(n, 1))

y = -1 + A + xbeta + np.random.normal(0, 1, size=(n, 1))


In [12]:
def save_temporary_data_for_R(yhat1, yhat0, ps, A, y, file_name):
    n = A.shape[0]
    data = pd.DataFrame(columns=['ps','yhat1','yhat0', 'A', 'y'], index=[c for c in range(n)])
    data['ps'] = ps
    data['yhat1'] = yhat1
    data['yhat0'] = yhat0
    data['A'] = A
    data['y'] = y
    
    data.to_csv(file_name+'.csv', index=False)
    
    return

def read_temporary_data_in_R(file_name):
    
    _ = ro.r(f"data_r <- read.csv('{file_name}.csv')")

    _ = ro.r('ps = matrix(data_r[, 1])')
    _ = ro.r('yhat1 = data_r[, 2]')
    _ = ro.r('yhat0 = data_r[, 3]')
    _ = ro.r('A = data_r[, 4]')
    _ = ro.r('y = data_r[, 5]')
    return


In [13]:
def run_MR_in_R():
    _ = ro.r("""
    MREst.mean <- function(y, A, Qs, ps)
    {

        J <- length(ps)
        K <- length(Qs)
        g.col <- J + K 

        n <- length(y)

        m <- sum(A) # number of observed subjects

        g.hat <- matrix(0, n, g.col)

        Qs = do.call(cbind, Qs)
        ps = do.call(cbind, ps)
        g.hat = cbind(Qs, ps)

        g.hat <- scale(g.hat, center = TRUE, scale = FALSE)[A == 1, ]

        # define the function to be minimized
        Fn <- function(rho, ghat){ -sum(log(1 + ghat %*% rho)) }
        Grd <- function(rho, ghat){ -colSums(ghat / c(1 + ghat %*% rho)) }

        # calculate the weights
        rho.hat <- constrOptim(theta = rep(0, g.col), 
                             f = Fn, 
                             grad = Grd, 
                             ui = g.hat, 
                             ci = rep(1 / m - 1, m), 
                             ghat = g.hat
                             )$par

        wts <- c(1 / m / (1 + g.hat %*% rho.hat))

        wts <- wts / sum(wts)

        estimate <- sum(y[A == 1] * wts)

        return(list(estimate = estimate, weights = wts))

    }
    """)
    return
    

In [14]:
def MR(y, A, yhat1, yhat0, ps, temporary_csv_name):
    
    run_MR_in_R()
    
    save_temporary_data_for_R(yhat1, yhat0, ps, A, y, file_name=temporary_csv_name)
    read_temporary_data_in_R(temporary_csv_name)
    
    uuu = importr("MultiRobust")
    ro.r('library("MultiRobust")')

    ro.r('result <- MREst.mean(y = y, A=A, Qs= list(yhat1), ps=list(ps))')

    beta1 = np.array(ro.r("result$estimate"))[0]

    ro.r('result <- MREst.mean(y = y, A=1-A, Qs= list(yhat0), ps=list(1-ps))')
    beta0 = np.array(ro.r("result$estimate"))[0]

    return beta1 - beta0

In [15]:
MR(y=y, 
   A=A, 
   yhat1=-1 + 1 + xbeta + np.random.normal(0, 1, size=(n, 1)), 
   yhat0=-1 + xbeta + np.random.normal(0, 1, size=(n, 1)), 
   ps=pr, 
   temporary_csv_name='temporary-MR-data')

1.0345543041749632